In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb
os.chdir(d['py'])
from gmsPython import nestingTree
from valueShares import nestedShares
from mProduction import NestedCES

# Investment module

The investment module is constructed like the production module, but does not include any durables (which makes things a bit easier).

*Load data:*

In [2]:
t0 = 2019 # set baseline year
name = 'vGR' # specify name of the CGE version we are currently working on
db = GpyDB(os.path.join(d['data'], f'{name}_{t0}')) # Load 
ws = db.ws # run everything from the same ws

*Set up nesting tree:*

In [3]:
nest = nestingTree.AggTree(name = 'I', trees = {'I': nestingTree.Tree('I', tree = db('nestInvestment').to_list(), f = 'CES')}, ws = ws)
nest(namespace = {str(n)+'_input':n for n in db('n')})

*Define share parameters:*

In [4]:
v = nestedShares(nest, ws = ws)
db_vs = v(db) 

*Use share parameters and values to get initial values for intermediates:*

In [5]:
db['mu'] = db_vs('mu').xs(t0) 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

*Subset database to include investment sectors, extrapolate data:*

In [6]:
AggDB.subsetDB(db, db('s_i'))
[db.series.__delitem__(k) for k in ('vD','vTax', 'vD_dur','vD_depr','vD_inv', 'vS', 'pD_dur') if k in db.symbols];
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

*Initialize and run:*

In [7]:
m = NestedCES(nest, partial = True)
m.initStuff(db = db)
soldb = m.jSolve(5, state = 'C', ϕ = .5) # solve calibration with 5 steps and nonlinear grid

*Overwrite solution:*

In [8]:
[m.db.__setitem__(k, soldb[k]) for k in m.db.getTypes(['var']) if k in soldb.symbols]; # use solution database
m.db.mergeInternal()
m.solve(state = 'B') # test solve of baseline